<a href="https://colab.research.google.com/github/orutega/thermal_analysis/blob/TEC_model/Thermal_Node_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#熱節点解析プログラム

##Googleドライブでフォルダを参照するために必要な処理

In [40]:
# Google Drive用　Google Drive上のファイルを使用するので，マウントするための処理
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##インポート

In [0]:
import numpy as np
import sympy as sp
import pandas as pd
import csv

##熱節点の情報をインポート

In [48]:
import csv

with open('/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv') as f:
    print(f.read())

# csv_file = open('/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv', "r", encoding="ms932", errors="", newline="" )
# #リスト形式
# f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
# print(f.reader())

Thermal_Conduction_Conductance_Matrix,,,,,,
K_ij,node1,node2,node3,node4,node5,node6
node1,N/A,,0,0,0,0
node2,,N/A,,,,
node3,,,N/A,,,
node4,,,,N/A,,
node5,,,,,N/A,
node6,,,,,,N/A
Thermal_Radiation_Conductance_Matrix,,,,,,
R_ij,node1,node2,node3,node4,node5,node6
node1,N/A,,,,,
node2,,N/A,,,,
node3,,,N/A,,,
node4,,,,N/A,,
node5,,,,,N/A,
node6,,,,,,N/A



In [32]:
csv_input = pd.read_csv(filepath_or_buffer='/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv', encoding="ms932", sep=",")
# インプットの項目数（行数 * カラム数）を返却します。
print(csv_input)

   Thermal_Conduction_Conductance_Matrix Unnamed: 1  ... Unnamed: 5 Unnamed: 6
0                                   K_ij      node1  ...      node5      node6
1                                  node1        NaN  ...          0          0
2                                  node2        NaN  ...        NaN        NaN
3                                  node3        NaN  ...        NaN        NaN
4                                  node4        NaN  ...        NaN        NaN
5                                  node5        NaN  ...        NaN        NaN
6                                  node6        NaN  ...        NaN        NaN
7   Thermal_Radiation_Conductance_Matrix        NaN  ...        NaN        NaN
8                                   R_ij      node1  ...      node5      node6
9                                  node1        NaN  ...        NaN        NaN
10                                 node2        NaN  ...        NaN        NaN
11                                 node3        NaN 

In [0]:
# T_state : T_iを状態変数ベクトルとして, tを時間とする。Node_const_ijは各Node
# Cond_N : ノード間の熱伝導コンダクタンスと熱輻射コンダクタンスを格納するクラス
# Spec_N : ノードの熱的な特性や発熱量などを格納するクラス（TECの旧熱量などはこの中に入る）
def thermal_equilibrium_equation(T_state, t, Cond_N, Spec_N):
  

## 熱伝導係数を保持するクラス

In [0]:
class Cond:
  def __init__(self, K_ij, R_ij):
    self.K_ij = K_ij
    self.R_ij = R_ij


## 各ノードの熱入出力や熱特性を保持するクラス

In [0]:
# Initialize Cond class class matrix
def init_cond(k_mat, r_mat):
  Cond_N = [[Cond(0,0) for i in range(len(k_mat[0]))] for j in range(len(k_mat[0]))]
  for Cond_N_i , k_i, r_i in zip(Cond_N, k_mat, r_mat):
    for Cond_N_ij, k_ij, r_ij in zip(Cond_N_i, k_i, r_i):
      Cond_N_ij.K_ij = k_ij
      Cond_N_ij.R_ij = r_ij
  return Cond_N

In [29]:
a = np.arange(9).reshape(3,3)
b = np.arange(9, 18).reshape(3,3)
Cond_N = init_cond(a, b)
print(Cond_N[1][0].K_ij)
print(b)

3
[[ 9 10 11]
 [12 13 14]
 [15 16 17]]


##TEC(Thermal Electric Cooler)のモデル化

\begin{align}
Q_c &= S_m \cdot T_c \cdot I - \frac{1}{2} \cdot R_m \cdot I^2 -K_m \cdot \Delta T \\
Q_h &= Q_c + V \cdot I \\
V &= S_m \cdot \Delta T + I \cdot R_m
\end{align}

\begin{align}
  & I : \text{Electric current passing through the Peltier} && [A] \\
  &Q_c  : \text{Absorbed power on the Peltier cold side} && [W] \\
  &Q_h : \text{Power rejected by the Peltier hot side} &&[W]\\
  &\Pi_{ab} : \text{Peltier effect module coefficient} &&[W]\\
  &S_m : \text{Seebeck effect module coefficient} &&[-]\\
  &K_m : \text{Thermal conductance of the module} &&[-]\\
  &R_m : \text{Electric resistance of the module} &&[\Omega]\\
  &T_c : \text{Cold side temperature} &&[K] \\
  &T_h : \text{Hot side temperature} &&[K] \\
  &\Delta T = T_h - T_c : \text{Temperature difference} &&[K]\\
\end{align}